# Домашнее задание 4

В последнем практическом задании мы потренируемся в построении VaR- и ES-кривой для небольшого портфеля. Каждый актив (акцию или валюту) далее будем называть инструментом.

### 1) Загрузка данных (бесценно)

* Представьте, что вы инвестор и собираете инвестиционный портфель. 

* Загрузите данные по трём любым акциям и двум любым валютам за последние 10 лет. Акции и валюты выберите на ваш вкус, но интереснее будет взять что-то кроме евро/доллара/акций гугла.

* Можно использовать YachooFinance либо другие источники при желании.

* Переведите цены в доходности. Можно использовать разности или процентные изменения на выбор.



In [1]:
# ༼ つ ◕_◕ ༽つ

### 2. Визуализация.

#### 2.1 (0.5 балла)
Изобразите парные диаграммы рассеяния (scatterplot) для всех комбинаций инструментов. В каких парах наблюдается связь, а в каких нет? Кратко интерпретируйте зависимость по одному примеру из предыдущего вопроса (один где есть зависимость, другой где нет). Если во всех парах есть зависимость (или во всех нет), то достаточно одного примера. 

In [2]:
# ༼ つ ◕_◕ ༽つ

#### 2.2 (1 балл)
Изобразите гистограмму распределения каждого инструмента, наложив на неё оценённое нормальное распределение. Прокомментируйте характеристики распределения доходностей.

In [ ]:
# ༼ つ ◕_◕ ༽つ

#### 2.3 (0.5 балла)
В качестве весов портфеля используйте веса на момент покупки (как в семинаре). Изобразите  гистограмму распределения доходностей портфеля, наложив на неё оценённое нормальное распределение. Прокомментируйте характеристики распределения доходностей.

In [ ]:
# ༼ つ ◕_◕ ༽つ


#### 2.4 (0.5 балла)
Придумайте, опишите и примените альтернативную стратегию взвешивания. Поясните, почему она имеет смысл.

In [ ]:
# ༼ つ ◕_◕ ༽つ

### 3. Анализ отдельных инструментов. (2 балла, по 0.4 за инструмент)

* Подберите для каждого инструмента наиболее подходящее маржинальное распределение. Можно использовать всё, что есть в scipy, а также garch-модель.
* Для каждого результата изобразите калибровочную кривую (qq-plot), а также наложите оценённую функцию плотности на гистограмму распределения доходности. Прокомментируйте графики.

Обратите внимание, что для GARCH-модели подход будет особенный в плане построения qq-plot и гистограммы. Как мы помним из формулы ниже, дисперсия у нас представлена умножением условной волатильности на распределение. Поэтому на qq-plot и гистограмме нужно будет визуализировать стандартизированные остатки против функции плотности $\epsilon$.

$$r_{t+1} = \mu_t  + \sqrt{h_t}\epsilon_t \text{  with  } \epsilon_t \sim N(0, 1)$$


NB! Постарайтесь подобрать распределение как можно точнее, от этого существенно зависит итоговый результат.


In [ ]:
# ༼ つ ◕_◕ ༽つ

### 4) Копулы
Мы знаем, что по-отдельности инструменты моделируются довольно легко. Однако так как мы рассматриваем портфель, акции и валюты могут быть скоррелированы. Особенно в случае, когда ваши акции взяты из похожих компаний. Поэтому нам бы хотелось иметь метод, который хорошо учитывает такие зависимости в совокупности. Копулы -- один из таких способов. Ваша последовательность действий будет выглядеть следующим образом:

Пусть мы имеем историю доходностей 5 инструментов за предыдущие 10 лет (условно 365*10 = 3650 наблюдений). Пусть мы хотим сделать прогноз VaR на один шаг вперёд. Представим данные в виде таблицы $3650\times5$

1. Используем найденные в предыдущем пункте распределения. Для каждого из инструментов прогоним доходности через фукнцию распределения. Для GARCH нужно будет предварительно получить стандартизированные остатки, так как распределение оценивается именно для них. Получим вектор той же длины, но уже в диапазоне [0, 1]. Получим таблицу аналогичной размерности, $3650\times5$. По сути, мы переходим в новое признаковое пространство, в котором нам легко оценить и просимулировать распределение с очень сложными зависимостями.

2. Оценим на этой матрице копулу. Выберем наилучшую копулу по AIC.

3. Произведём симуляции из копулы. Так как копула будет иметь размерность 5, то выходная матрица симуляций будет иметь размер $n\_sim\times5$. Ожидается, что компоненты этой симуляции будут коррелированы соответственно исходным данным.

4. Для каждого из пяти инструментов в симуляции проведём обратное преобразование. Для этого используем обратную функцию распределения. Для GARCH нужно будет дополнительно использовать прогнозы матожидания и дисперсии. Получим всё также матрицу $n\_sim\times5$.

5. Далее для подсчёта VaR можно просто взвесить доходности отдельных компонент обратно в портфель и взять от этого квантиль. Для подсчёта ES можно взять математичечкое ожидание доходностей портфеля, которые меньше VaR. Используйте 5%-й VaR.

Используйте одну копулу. Для простоты давайте ограничимся [Архимедовыми копулами](https://copulae.readthedocs.io/en/latest/api_reference/copulae/archimedean/index.html). Они имеют минимум параметров и быстро оцениваются. Не пытайтесь выполнить это задание с эллиптическими копулами (гауссова, стьюдента). Они оцениваются очень долго и вы устанете считать.

Начнём с простого и будем работать со всеми данными сразу. 


#### 4.1 (1 балл)
Проведите преобразование, переводящее доходности на единичный куб. Воспользуйтесь функциями распределения из пункта 4. Опять же, обратите внимание, что GARCH необходимо обрабатывать отдельно и работать с распределением стандартизированных остатков.

In [1]:
# ༼ つ ◕_◕ ༽つ

#### 4.2 (1 балл) 

Используйте пакет [copulae](https://pypi.org/project/copulae/). Подберите по AIC Архимедовы копулы. Выберите одну из двух опций:
1. Выберите лучшую копулу на всех доходностях.  (0.5 балла)
2. Выберите две отдельные копулы. Одна на валюты, одна на акции. (1 балл)


In [ ]:
# ༼ つ ◕_◕ ༽つ

#### 4.3 (1 балл)
Просимулируйте с помощью копулы ваши инструменты. Для любой пары инструментов нарисуйте четыре scatter-plot в ряд:

1. Исходные доходности
2. Доходности, переведённые на единичный куб
3. Просимулированные доходности на единичном кубе
4. Просимулированные доходности, переведённые в исходные величины

Графики 1/4 и 2/3 должны иметь примерно одинаковые зависимости.

### 5) VaR и ES (1 балл)

С помощью выбранных распределений и копул вычислите VaR-кривую и ES-кривую. Можно ориентироваться на функцию с семинара 12. Модифицируйте её для подсчёта ES. Если вы использовали альтернативную стратегию взвешивания, также учтите это.

In [ ]:
import numpy as np
import pandas as pd 

def calc_VaR(r, sampling_fun, w, L_history=252, level=0.05):
    VaR = np.full(r.shape[0], np.nan)
    ES = np.full(r.shape[0], np.nan)
    
    for i in range(L_history, r.shape[0]):
        history = r[i - L_history: i]
        
        try:
            return_simulations = sampling_fun(history)
            portfolio_simulated_returns = np.dot(return_simulations.values, w)

            VaR[i] = np.quantile(portfolio_simulated_returns, level)
            ES[i] = #TODO
            
        except:
            VaR[i] = VaR[i-1]
            ES[i] = ES[i-1]
        
    return pd.Series(data=VaR, index=r.index, name=sampling_fun.__name__)

In [ ]:
# ༼ つ ◕_◕ ༽つ

### 6) Визуализация (0.5 балла)

Визуализируйте VaR и ES на графике с доходностями.

In [ ]:
# ༼ つ ◕_◕ ༽つ

### 7) Бектестинг. (1 балл)

Проведите тесты на покрытие и независимость для VAR-кривой. Добейтесь того, чтобы тесты успешно проходились, без этого пункт не засчитывается. Если потребуется, вернитесь к подбору частных распределений.

In [ ]:
# ༼ つ ◕_◕ ༽つ

##### Рубрика "как вам домашка?" (0.1 балла)

Пройдите короткий опрос. Это действительно важно. https://forms.gle/w3sV453spERTbGvr7